In [20]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from itertools import product
import os

In [21]:
train_df = pd.read_csv('/kaggle/input/tweepfake-dataset/train.csv')
test_df = pd.read_csv('/kaggle/input/tweepfake-dataset/test.csv')
val_df = pd.read_csv('/kaggle/input/tweepfake-dataset/validation.csv')

In [22]:
train_df['label'] = train_df['account.type'].apply(lambda x: 1 if x == 'human' else 0)
test_df['label'] = test_df['account.type'].apply(lambda x: 1 if x == 'human' else 0)
val_df['label'] = val_df['account.type'].apply(lambda x: 1 if x == 'human' else 0)

In [23]:
model_checkpoint = "albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/20712 [00:00<?, ? examples/s]

Map:   0%|          | 0/2558 [00:00<?, ? examples/s]

Map:   0%|          | 0/2302 [00:00<?, ? examples/s]

In [24]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}


In [25]:
param_combinations = [
    [2e-5, 16, 3, 0.01, (0.9, 0.999)],  # Combination 1: Baseline Standard
    [5e-5, 32, 3, 0.05, (0.9, 0.999)],  # Combination 2: Higher Learning Rate, Larger Batch
    [1e-5, 8, 3, 0.1, (0.95, 0.98)],    # Combination 3: Lower Learning Rate, Smaller Batch
    [2e-5, 16, 2, 0.1, (0.95, 0.98)]    # Combination 4: Regularized Model with Lower Epochs
]

# Result storage
results = []

# Disable WandB logging
os.environ["WANDB_DISABLED"] = "true"

# Iterate through each combination and run the experiment
for lr, batch_size, epochs, weight_decay, betas in param_combinations:
    print(f"Running experiment with lr={lr}, batch_size={batch_size}, epochs={epochs}, weight_decay={weight_decay}, betas={betas}")
    
    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f"./results/lr{lr}_bs{batch_size}_ep{epochs}_wd{weight_decay}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=weight_decay,
        adam_beta1=betas[0],
        adam_beta2=betas[1],
        logging_dir="./logs",
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy"
    )
    
    # Initialize model and trainer
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    
    # Train the model
    trainer.train()
    
    # Evaluate on test set
    eval_results = trainer.evaluate(test_dataset)
    results.append({
        "learning_rate": lr,
        "batch_size": batch_size,
        "epochs": epochs,
        "weight_decay": weight_decay,
        "adam_betas": betas,
        "accuracy": eval_results["eval_accuracy"]
    })
    
    # Save results to file
    pd.DataFrame(results).to_csv("hyperparameter_results.csv", index=False)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Running experiment with lr=2e-05, batch_size=16, epochs=3, weight_decay=0.01, betas=(0.9, 0.999)


model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.314300,0.310921,0.864031
2,0.171900,0.291609,0.878801
3,0.144600,0.332756,0.885752


...

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running experiment with lr=2e-05, batch_size=16, epochs=2, weight_decay=0.1, betas=(0.95, 0.98)


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.278800,0.286714,0.873588
2,0.214700,0.277613,0.882276


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

In [26]:
# Load results and analyze
results_df = pd.read_csv("hyperparameter_results.csv")
best_params = results_df.loc[results_df['accuracy'].idxmax()]
print("Best Parameters:")
print(best_params)

# Generate a summary table
print("All Results:")
print(results_df.sort_values(by="accuracy", ascending=False))


Best Parameters:
learning_rate         0.00001
batch_size                  8
epochs                      3
weight_decay              0.1
adam_betas       (0.95, 0.98)
accuracy             0.893667
Name: 2, dtype: object
All Results:
   learning_rate  batch_size  epochs  weight_decay    adam_betas  accuracy
2        0.00001           8       3          0.10  (0.95, 0.98)  0.893667
3        0.00002          16       2          0.10  (0.95, 0.98)  0.892103
1        0.00005          32       3          0.05  (0.9, 0.999)  0.886630
0        0.00002          16       3          0.01  (0.9, 0.999)  0.886239
